In [51]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import json
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [34]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bidding_imbalance_price = pd.read_csv('bidding_training_predictions_imbalance_price.csv')
df_bidding_imbalance_price = df_bidding_imbalance_price[["timestamp_utc","imbalance_price_predictions"]]
df_bidding_imbalance_price.timestamp_utc = pd.to_datetime(df_bidding_imbalance_price.timestamp_utc)
df_bidding_day_ahead_price = pd.read_csv('bidding_training_predictions_day_ahead_price.csv')
df_bidding_day_ahead_price = df_bidding_day_ahead_price[["timestamp_utc","day_ahead_price_predictions"]]
df_bidding_day_ahead_price.timestamp_utc = pd.to_datetime(df_bidding_day_ahead_price.timestamp_utc)
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
# df_day_ahead = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/day_ahead_price.csv')
# df_imbalance = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/imbalance_price.csv')

In [35]:
df_bbidding.timestamp_utc = pd.to_datetime(df_bbidding.timestamp_utc)
df_bbidding = df_bbidding.merge(df_bidding_imbalance_price, on="timestamp_utc")
df_bbidding = df_bbidding.merge(df_bidding_day_ahead_price, on="timestamp_utc")

In [36]:
df_bbidding

,timestamp_utc,Mean_SolarRadiation_dwd,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_RW_dwd_Mean_1h,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Efficiency_dwd_mean,Panel_Efficiency_dwd_std,Panel_Temperature_dwd_mean,...,imbalance_price,settlement_date,settlement_period,data_provider,price_y,volume,day_ahead_price,market_price,imbalance_price_predictions,day_ahead_price_predictions
0,2021-02-22 23:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.192863,0.000717,7.134149,...,39.86000,2021-02-22,47,APXMIDP,39.86,850.50,40.74,39.86,51.824390,43.437900
1,2021-02-22 23:30:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.193050,0.000716,6.875409,...,25.88000,2021-02-22,48,APXMIDP,25.88,755.65,40.74,25.88,51.953340,43.062424
2,2021-02-23 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.192984,0.000676,6.966046,...,31.95000,2021-02-23,1,APXMIDP,31.95,821.25,35.25,31.95,49.136684,43.795162
3,2021-02-23 00:30:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.193155,0.000679,6.728668,...,32.18992,2021-02-23,2,APXMIDP,32.09,852.25,35.25,32.09,49.771446,43.883200
4,2021-02-23 01:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.193326,0.000696,6.491290,...,30.55000,2021-02-23,3,APXMIDP,30.75,789.00,32.74,30.75,48.788677,44.415764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42184,2023-08-25 23:30:00+00:00,0.009818,0.009818,0.004048,-0.001721,-0.003729,0.004615,0.187994,0.000419,13.896639,...,66.01000,2023-08-26,2,APXMIDP,83.91,1201.85,84.41,83.91,85.434860,97.826805
42185,2023-08-26 00:00:00+00:00,0.021356,0.021356,0.015587,0.009818,-0.001721,0.017088,0.188246,0.000398,13.547243,...,66.01000,2023-08-26,3,APXMIDP,82.13,1066.95,79.96,82.13,86.159660,96.985740
42186,2023-08-26 00:30:00+00:00,0.012875,0.012875,0.017116,0.021356,0.009818,-0.000814,0.188469,0.000386,13.237374,...,66.01000,2023-08-26,4,APXMIDP,78.76,997.60,79.96,78.76,85.281170,96.886720
42187,2023-08-26 01:00:00+00:00,0.004395,0.004395,0.008635,0.012875,0.021356,-0.018716,0.188692,0.000381,12.927505,...,66.01000,2023-08-26,5,APXMIDP,77.47,913.25,73.84,77.47,96.748120,101.025780


In [38]:
df_bbidding.Target_MW

0         0.000
1         0.000
2         3.968
3        41.184
4        77.831
          ...  
42184    23.648
42185    10.865
42186    10.505
42187     4.486
42188     6.580
Name: Target_MW, Length: 42189, dtype: float64

In [39]:
df_bbidding["Revenue_50"] = df_bbidding["day_ahead_price"] * df_bbidding["5"]+(df_bbidding["Target_MW"]-df_bbidding["5"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["5"]))

In [40]:
df_bbidding.Revenue_50.mean()

25492.389316630168

50% quantil gibt historisch 25437.310022730526 pro periode

In [41]:
df_bbidding["Revenue_40"] = df_bbidding["day_ahead_price"] * df_bbidding["4"]+(df_bbidding["Target_MW"]-df_bbidding["4"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["4"]))
df_bbidding["Revenue_30"] = df_bbidding["day_ahead_price"] * df_bbidding["3"]+(df_bbidding["Target_MW"]-df_bbidding["3"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["3"]))
df_bbidding["Revenue_20"] = df_bbidding["day_ahead_price"] * df_bbidding["2"]+(df_bbidding["Target_MW"]-df_bbidding["2"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["2"]))
df_bbidding["Revenue_10"] = df_bbidding["day_ahead_price"] * df_bbidding["1"]+(df_bbidding["Target_MW"]-df_bbidding["1"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["1"]))
df_bbidding["Revenue_60"] = df_bbidding["day_ahead_price"] * df_bbidding["6"]+(df_bbidding["Target_MW"]-df_bbidding["6"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["6"]))
df_bbidding["Revenue_50"] = df_bbidding["day_ahead_price"] * df_bbidding["5"]+(df_bbidding["Target_MW"]-df_bbidding["5"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["5"]))
df_bbidding["Revenue_70"] = df_bbidding["day_ahead_price"] * df_bbidding["7"]+(df_bbidding["Target_MW"]-df_bbidding["7"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["7"]))
df_bbidding["Revenue_80"] = df_bbidding["day_ahead_price"] * df_bbidding["8"]+(df_bbidding["Target_MW"]-df_bbidding["8"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["8"]))
df_bbidding["Revenue_90"] = df_bbidding["day_ahead_price"] * df_bbidding["9"]+(df_bbidding["Target_MW"]-df_bbidding["9"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["9"]))

In [42]:
for i in range(1,10):
    print("Revenue_"+str(i)+"0")
    print(df_bbidding["Revenue_"+str(i)+"0"].mean())

Revenue_10
35062.207379775966
Revenue_20
32821.24134246784
Revenue_30
30371.60110729176
Revenue_40
27229.629375078024
Revenue_50
25492.389316630168
Revenue_60
22691.284657256125
Revenue_70
19767.671762280588
Revenue_80
15539.252709018074
Revenue_90
7629.842740345747


In [ ]:
df_bbidding.columns.to_list()

In [44]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price']
    Target_MW = row['Target_MW']
    imbalance_price = row['imbalance_price']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_optimal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))



Mit optimierung 56488.5046925057 pro periode

In [45]:
df_bbidding.revenue_optimal.mean()

56591.935789578005

In [46]:
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

Revenue mit estimateten values (LSTM) Mit 50% quantile

In [47]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['5']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))


In [48]:
df_bbidding.revenue_normal.mean()

26015.02479507701

LSTM 40% quantile

In [49]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['4']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))


In [50]:
df_bbidding.revenue_normal.mean()

27753.782314904725

On newer data

In [66]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [84]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [85]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new

,prediction_date,timestamp,1,2,3,4,5,6,7,8,9
0,2024-10-04,2024-10-03T22:00:00+00:00,1355,487,963,1544,1330,867,326,1402,1357
1,2024-10-04,2024-10-03T22:30:00+00:00,788,584,1148,1320,1007,565,923,797,487
2,2024-10-04,2024-10-03T23:00:00+00:00,571,748,1001,1460,740,1017,1533,597,991
3,2024-10-04,2024-10-03T23:30:00+00:00,1476,1194,1512,334,1054,1572,744,1497,463
4,2024-10-04,2024-10-04T00:00:00+00:00,1352,1390,912,662,1505,1093,814,487,1340
...,...,...,...,...,...,...,...,...,...,...,...
811,2024-10-19,2024-10-19T19:30:00+00:00,3,7,23,32,45,57,77,93,125
812,2024-10-19,2024-10-19T20:00:00+00:00,0,11,25,36,42,60,82,103,132
813,2024-10-19,2024-10-19T20:30:00+00:00,0,16,27,39,44,64,89,106,140
814,2024-10-19,2024-10-19T21:00:00+00:00,0,11,29,42,51,75,97,113,151


In [86]:
df_api_new["timestamp"] = pd.to_datetime(df_api_new["timestamp"])
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.day/7)


In [87]:
df_api_new_merged1 = df_api_new_merged[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag48h"] = df_api_new_merged1["market_price"].shift(96)
df_api_new_merged1.loc[:,"imbalance_price_lag48h"] = df_api_new_merged1["imbalance_price"].shift(96)
df_api_new_merged1.loc[:,"day_ahead_price_lag48h"] = df_api_new_merged1["day_ahead_price"].shift(96)
df_api_new_merged1.loc[:,"volume_lag48h"] = df_api_new_merged1["volume"].shift(96)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1 = df_api_new_merged1.groupby("timestamp_utc").last().reset_index()


In [88]:
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')
wind_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/wind_total_production.csv')
solar_total.generation_mw = solar_total.generation_mw * 0.5
wind_total.generation_mw = wind_total.generation_mw * 0.5 - wind_total.boa
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
wind_total.timestamp_utc = pd.to_datetime(wind_total.timestamp_utc)

In [91]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h,generation_mw_x,installed_capacity_mwp,capacity_mwp,settlement_date,settlement_period_y,boa,generation_mw_y
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19,17,15,...,50.0,78.47,1035.65,0.0,2956.745251,2779.541874,2024-10-07,47,0.0,236.742
1,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19,17,15,...,50.0,78.47,1035.65,0.0,2956.745251,2779.542388,2024-10-07,47,0.0,236.742
2,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19,17,15,...,93.5,71.36,1378.55,0.0,2956.745251,2779.541862,2024-10-07,48,0.0,154.032
3,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19,17,15,...,93.5,71.36,1378.55,0.0,2956.745251,2779.542388,2024-10-07,48,0.0,154.032
4,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19,17,15,...,93.5,71.36,1539.90,0.0,2956.745251,2779.541852,2024-10-08,1,0.0,124.472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,0.258819,-0.900969,19,17,14,...,66.0,89.55,1265.00,0.0,2956.745251,2779.347644,2024-10-10,42,0.0,361.670
99,2024-10-10 20:00:00+00:00,93.04,79.14,1992.70,43.0,0.500000,-0.900969,19,16,14,...,66.0,87.15,1112.60,0.0,2956.745251,2779.347643,2024-10-10,43,0.0,410.810
100,2024-10-10 20:30:00+00:00,89.29,79.14,2284.50,44.0,0.500000,-0.900969,19,16,14,...,66.0,87.15,1240.55,0.0,2956.745251,2779.347622,2024-10-10,44,0.0,423.910
101,2024-10-10 21:00:00+00:00,78.26,91.45,1793.20,45.0,0.707107,-0.900969,19,17,14,...,65.0,81.06,1297.45,0.0,2956.745251,2779.347620,2024-10-10,45,0.0,417.860


In [93]:
df_api_new_merged2 = pd.merge(df_api_new_merged1,solar_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = pd.merge(df_api_new_merged2,wind_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = df_api_new_merged2.rename(columns={
    "generation_mw_x": "generation_solar",
    "generation_mw_y": "generation_wind"
})
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h,generation_solar,installed_capacity_mwp,capacity_mwp,settlement_date,settlement_period_y,boa,generation_wind
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19,17,15,...,50.0,78.47,1035.65,0.0,2956.745251,2779.541874,2024-10-07,47,0.0,236.742
1,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19,17,15,...,50.0,78.47,1035.65,0.0,2956.745251,2779.542388,2024-10-07,47,0.0,236.742
2,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19,17,15,...,93.5,71.36,1378.55,0.0,2956.745251,2779.541862,2024-10-07,48,0.0,154.032
3,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19,17,15,...,93.5,71.36,1378.55,0.0,2956.745251,2779.542388,2024-10-07,48,0.0,154.032
4,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19,17,15,...,93.5,71.36,1539.90,0.0,2956.745251,2779.541852,2024-10-08,1,0.0,124.472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,0.258819,-0.900969,19,17,14,...,66.0,89.55,1265.00,0.0,2956.745251,2779.347644,2024-10-10,42,0.0,361.670
99,2024-10-10 20:00:00+00:00,93.04,79.14,1992.70,43.0,0.500000,-0.900969,19,16,14,...,66.0,87.15,1112.60,0.0,2956.745251,2779.347643,2024-10-10,43,0.0,410.810
100,2024-10-10 20:30:00+00:00,89.29,79.14,2284.50,44.0,0.500000,-0.900969,19,16,14,...,66.0,87.15,1240.55,0.0,2956.745251,2779.347622,2024-10-10,44,0.0,423.910
101,2024-10-10 21:00:00+00:00,78.26,91.45,1793.20,45.0,0.707107,-0.900969,19,17,14,...,65.0,81.06,1297.45,0.0,2956.745251,2779.347620,2024-10-10,45,0.0,417.860


In [94]:
import pickle
df_api_new_merged2_X = df_api_new_merged2[["market_price_lag48h","imbalance_price_lag48h","day_ahead_price_lag48h","volume_lag48h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values

scaler_path = "LSTM_imbalance_scaler.pkl"
# Laden des StandardScalers aus der Datei
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

# Skalieren der Daten
df_api_new_merged2_X_scaled = scaler.transform(df_api_new_merged2_X)

# Konvertieren der Daten in PyTorch-Tensoren
X_test = torch.tensor(df_api_new_merged2_X_scaled, dtype=torch.float32)
X_test = X_test.unsqueeze(1)  # Adds a sequence length dimension

In [95]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_imbalance_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["imvalance_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_28960\1576695596.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_imbalance_p

In [96]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["day_ahead_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_28960\823899303.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_pr

In [98]:
df_api_new_merged2["Total_MW"] = df_api_new_merged2["generation_solar"] + df_api_new_merged2["generation_wind"]

In [100]:
df_api_new_merged2.columns

Index(['timestamp_utc', 'market_price', 'day_ahead_price', 'volume',
       'settlement_period_x', 'cos_hour', 'cos_day', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'imbalance_price', 'market_price_lag48h',
       'imbalance_price_lag48h', 'day_ahead_price_lag48h', 'volume_lag48h',
       'generation_solar', 'installed_capacity_mwp', 'capacity_mwp',
       'settlement_date', 'settlement_period_y', 'boa', 'generation_wind',
       'imvalance_price_predictions', 'day_ahead_price_predictions',
       'Total_MW'],
      dtype='object')

In [101]:
df_api_new_merged2["Revenue_40"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["4"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["4"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["4"]))
df_api_new_merged2["Revenue_30"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["3"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["3"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["3"]))
df_api_new_merged2["Revenue_20"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["2"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["2"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["2"]))
df_api_new_merged2["Revenue_10"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["1"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["1"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["1"]))
df_api_new_merged2["Revenue_60"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["6"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["6"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["6"]))
df_api_new_merged2["Revenue_50"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["5"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["5"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["5"]))
df_api_new_merged2["Revenue_70"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["7"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["7"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["7"]))
df_api_new_merged2["Revenue_80"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["8"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["8"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["8"]))
df_api_new_merged2["Revenue_90"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["9"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["9"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["9"]))

In [103]:
for i in range(1,10):
    print("Revenue_"+str(i)+"0")
    print(df_api_new_merged2["Revenue_"+str(i)+"0"].mean())

Revenue_10
19762.317489870224
Revenue_20
19576.866366634877
Revenue_30
19397.694989130057
Revenue_40
19193.734100690006
Revenue_50
19013.622173722826
Revenue_60
18818.49142995129
Revenue_70
18623.900104798708
Revenue_80
18426.0589535053
Revenue_90
18237.802612376887


In [105]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['5']
    imbalance_price = row['imvalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']))


In [106]:
df_bbidding.revenue_normal.mean()

27753.782314904725

monte carlo: 

In [ ]:
import numpy as np

# Define revenue calculation function
def calculate_revenue(trade, actual, DAP, SSP):
    return trade * DAP + (actual - trade) * (SSP - 0.07 * (actual - trade))

# Monte Carlo simulation to estimate expected revenue
def monte_carlo_simulation(trade_values, actual_distribution, DAP, SSP, num_samples=1000):
    expected_revenue = []
    
    # Sample from the probabilistic distribution (Monte Carlo simulation)
    sampled_actuals = actual_distribution(num_samples)
    
    for trade in trade_values:
        # Calculate revenue for each sample and trade value
        revenues = [calculate_revenue(trade, actual, DAP, SSP) for actual in sampled_actuals]
        # Store the average expected revenue for this trade value
        expected_revenue.append(np.mean(revenues))
    
    return trade_values, expected_revenue

# Example usage
# Define input values (these would be dynamic in practice)
DAP = 50  # Day-Ahead Price
SSP = 45  # System Sell Price

# Generate probabilistic forecast (e.g., Normal distribution with mean and std dev from your forecast)
actual_mean = 100  # Assume mean production is 100 MWh
actual_std = 10    # Assume a standard deviation of 10 MWh
actual_distribution = lambda n: np.random.normal(actual_mean, actual_std, n)

# Define the range of Trade values to evaluate
trade_values = np.linspace(80, 120, 50)  # Evaluate Trade between 80 and 120 MWh

# Run Monte Carlo simulation
trade_values, expected_revenues = monte_carlo_simulation(trade_values, actual_distribution, DAP, SSP)

# Find the optimal trade value that maximizes expected revenue
optimal_trade = trade_values[np.argmax(expected_revenues)]

print(f"Optimal Trade Value: {optimal_trade} MWh")


1. Dynamic Programming Approach


In [ ]:
for t in range(T-1, -1, -1):  # Start at last time step, move backwards
    for s in state_space:  # For each state in the state space
        max_value = -inf
        for a in action_space:  # For each possible trade action
            next_state = transition_function(s, a)
            immediate_reward = revenue(s, a)
            future_value = gamma * V[next_state]  # Discounted future value
            total_value = immediate_reward + future_value
            if total_value > max_value:
                max_value = total_value
                best_action[s] = a  # Store the best action for state s
        V[s] = max_value
